In [1]:
import json
import numpy
from pandas.io.json import json_normalize
from time import time
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import scipy.stats as sp
from plotly import figure_factory as ff

In [2]:
df_business = pd.read_csv("yelp_academic_dataset_business.csv",encoding='latin-1') 

In [3]:
df_review=pd.read_csv("yelp_academic_dataset_review.csv",encoding='latin-1') 

In [4]:
df_user=pd.read_csv("yelp_academic_dataset_user.csv",encoding='latin-1') 

# Data filtering (If need Filtering)
## 1. Filter out the American cities from the review dataset

In [91]:
def city_filter(df_business,cities):
    return df_business[df_business.city.isin(cities)]
cities=['Toronto']#['Las Vegas', 'Phoenix', 'Scottsdale','Charlotte', 'Pittsburgh']
df_abusiness=city_filter(df_business,cities)

## 2. Filter out users with less than M reviews.

In [ ]:
def user_filter(df_user,min_review):
    return df_user[df_user.review_count>min_review]
df_userf=user_filter(df_user,20)

## 3. Filter out business with less than X reviews

In [ ]:
def business_filter(df_business,min_review):
    return df_business[df_business.review_count>min_review]
df_businessf=business_filter(df_abusiness,50)

In [5]:
#Return df_abusiness and df_abr
#df_abu
def bus_review_combine(df_business,df_review):
    #combine the two tables 
    #df_abr: df_american business and review combined table
    #df_abr=pd.merge(left=df_review,right=df_abusiness, how='right', left_on='business_id', right_on='business_id')
    df_business=df_business.rename(columns={'stars':'bus_stars'})
    df_abr=pd.merge(left=df_review,right=df_business,how='right', on='business_id')
    df_abr=df_abr[['date','business_id','user_id','stars']]
    return df_abr

# Start from here if Filtering is not needed

In [6]:
#Use this line if filtering is needed:
#df_abr=bus_review_combine(df_abusiness,df_review)
#Use this line if filtering is NOT needed:
df_abr=bus_review_combine(df_abusiness,df_review)


In [83]:
def a_rating_creat(aggregations, base_table):
    dup=base_table.copy()
    base_table=base_table.groupby(['business_id','date','user_id'])['stars'].agg(aggregations)
    #a=base_table.groupby(['business_id','date','user_id']).agg(aggregations)
    base_table=base_table.drop(base_table.index[0])
    base_table=base_table.reset_index()
    base_table.columns = base_table.columns.get_level_values(0)
    base_table.columns=['business_id','date','user_id','sum','count']
    t=base_table.groupby(['business_id','date','user_id']).sum().groupby(level=[0]).cumsum()
    t=t.reset_index()
    base_table['cumcount']=t['count']
    #calculate the aggregated ratings for each date of each business
    l=[]
    for i in range(0,len(base_table)):
        if i==0:
            j=1
            while base_table.business_id[i]==base_table.business_id[i+j-1]:
                if base_table.business_id[i]==base_table.business_id[i+j-1]:
                    l.append(sum(base_table['sum'][i:i+j])/sum(base_table['count'][i:i+j]))
                    #print(i,i+a)
                    j+=1

        elif base_table.business_id[i]!=base_table.business_id[i-1]:
            j=1
            while base_table.business_id[i]==base_table.business_id[i+j-1]:
                l.append(sum(base_table['sum'][i:i+j])/sum(base_table['count'][i:i+j]))
                #print(i,i+a)
                if (i+j)==len(base_table):
                #if (i+j)<10000-1:
                    break
                else:
                    j+=1
    #add the cumulative average list to the table, call the column 'cumav' as cumulative average
    base_table['cumav']=l
    df_rating=pd.merge(left=base_table,right=dup, how='left',on=['business_id','date','user_id'])
    return df_rating

In [96]:
aggregations={
    'stars':{
        'sum':'sum',
        'count':'count'
    }
}

In [119]:
df_rating=a_rating_creat(aggregations,df_abr)

In [120]:
def round_of_rating(row):
    return round(row.cumav * 2) / 2

In [121]:
df_rating['cum_rating']=df_rating.apply(lambda row: round_of_rating(row),axis=1)

In [122]:
del df_rating['cumav']

## 2. measure the delta

### 1) measure absolute delta

In [123]:
df_rating['delta']=df_rating.stars-df_rating.cum_rating

### 2) measure normalised delta

In [124]:
df_rating['norm_delta']=df_rating.delta/df_rating.cum_rating

## 3. User rating order by dates

In [125]:
df_rating=df_rating.sort('date')

/Users/stevezhang/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [126]:
i=df_rating.groupby('user_id')['stars'].apply(list).reset_index()
df_rating=pd.merge(left=i,right=df_rating,on='user_id',how='right')

In [127]:
df_rating=df_rating.rename(columns={'stars_x':'ratings','stars_y':'stars'})

In [103]:
df_rating.to_csv('CA_rating_table.csv')

In [128]:
df_rating.to_csv('basic_rating_table.csv')

In [70]:
df_rating.to_csv('american_rating_table.csv')